In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
x = tf.constant([2., 1., 0.1])
layer = layers.Softmax(axis=-1)
layer(x)

<tf.Tensor: id=2, shape=(3,), dtype=float32, numpy=array([0.6590012, 0.242433 , 0.0985659], dtype=float32)>

In [4]:
import os
os.path.abspath('.')

'D:\\gitprojects\\tensorflow_learning'

# Packaging layers

In [5]:
from tensorflow.keras import Sequential

## Method 1

In [6]:
network = Sequential([
    layers.Dense(3, activation=None),
    layers.ReLU(),
    layers.Dense(2, activation=None),
    layers.ReLU()
])

In [7]:
x = tf.random.normal([4, 3])
network(x)

<tf.Tensor: id=62, shape=(4, 2), dtype=float32, numpy=
array([[1.2332035 , 1.1286494 ],
       [1.2638199 , 1.1566701 ],
       [0.61706537, 0.564749  ],
       [0.9920784 , 0.13879845]], dtype=float32)>

In [8]:
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  12        
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  8         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [9]:
network.trainable_variables

[<tf.Variable 'sequential/dense/kernel:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 0.1026566 , -0.19409537, -0.77417755],
        [-0.27476716,  0.5057335 ,  0.792434  ],
        [-0.99350977,  0.565011  , -0.48227954]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'sequential/dense_1/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.8262427 ,  0.75619173],
        [ 0.23996866, -0.23909664],
        [ 0.11702025, -0.16015768]], dtype=float32)>,
 <tf.Variable 'sequential/dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

## Method 2

In [10]:
layers_num = 2
network = Sequential([])
for _ in range(layers_num):
    network.add(layers.Dense(3))
    network.add(layers.ReLU())

network.build(input_shape=(None, 4))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              multiple                  15        
_________________________________________________________________
re_lu_2 (ReLU)               multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  12        
_________________________________________________________________
re_lu_3 (ReLU)               multiple                  0         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [11]:
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_2/kernel:0 (4, 3)
dense_2/bias:0 (3,)
dense_3/kernel:0 (3, 3)
dense_3/bias:0 (3,)


# Build, train and test the model

In [6]:
from tensorflow.keras import datasets, optimizers, losses

In [7]:
(x_train, y_train),(x_test, y_test) = datasets.mnist.load_data()

In [8]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## Build

In [15]:
network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])
network.build(input_shape=(None, 28*28))
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [16]:
network.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

## Train

In [ ]:
x_train = 2 * tf.convert_to_tensor(x_train, dtype=tf.float32)/255. -1
x_train = tf.reshape(x_train, (-1, 28*28))

y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_train = tf.one_hot(y_train, depth=10)

In [ ]:
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_db = train_db.shuffle(100).batch(512)

In [ ]:
x_test = 2 * tf.convert_to_tensor(x_test, dtype=tf.float32)/255. -1
x_test = tf.reshape(x_test, (-1, 28*28))

y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)
y_test = tf.one_hot(y_test, depth=10)

In [ ]:
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [ ]:
history = network.fit(x_train,y_train, epochs=5)

## Test and Evaluate

In [22]:
score = network.evaluate(x_test, y_test, batch_size=128)
score

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.26674724429249763, 0.9312]

# Save and import the model

## 1. By Tensors
### Save

In [25]:
network.save_weights('weights.ckpt')
print('saved weights')
del network

saved weights


### Import

In [26]:
network = Sequential([
    layers.Dense(256,activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])
network.build(input_shape=(None, 28*28))
network.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [27]:
network.load_weights('weights.ckpt')
print('loaded weights')

loaded weights


## 2. By network
### Save
Include structure and weights.

In [28]:
network.save(r'model.h5',include_optimizer=True)

In [29]:
del network

### Import

In [30]:
network = tf.keras.models.load_model('model.h5')

## 3. By SavedModel()
### Save

In [31]:
tf.keras.models.save_model(network, 'model_savedmodel', save_format='tf')
print('export saved model.')
del network

ValueError: Model <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002A785E7C518> cannot be saved because the input shapes have not been set. Usually, input shapes are automatically determined from calling .fit() or .predict(). To manually set the shapes, call model._set_inputs(inputs).

### Import

In [32]:
network = tf.keras.models.load_model( 'model_savedmodel')

OSError: SavedModel file does not exist at: model_savedmodel/{saved_model.pbtxt|saved_model.pb}

In [19]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, None, 256)         200960    
_________________________________________________________________
dense_6 (Dense)              (None, None, 128)         32896     
_________________________________________________________________
dense_7 (Dense)              (None, None, 64)          8256      
_________________________________________________________________
dense_8 (Dense)              (None, None, 32)          2080      
_________________________________________________________________
dense_9 (Dense)              (None, None, 10)          330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


# Customized model

## Customize layers
Example 1.1 : create a `layer` o = w * x.

In [33]:
class MyDense(layers.Layer):
    def __init__(self, input_dim, output_dim):
        super(MyDense, self).__init__()
        self.kernel = self.add_weight('w', [input_dim, output_dim], trainable=True)

In [34]:
net = MyDense(4,3)
net.variables, net.trainable_variables

([<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[-0.45781597,  0.05759579, -0.8975703 ],
         [ 0.5045341 , -0.4327979 ,  0.67372835],
         [ 0.3302629 , -0.42117172,  0.91816795],
         [ 0.50909865, -0.4464873 , -0.3630113 ]], dtype=float32)>],
 [<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[-0.45781597,  0.05759579, -0.8975703 ],
         [ 0.5045341 , -0.4327979 ,  0.67372835],
         [ 0.3302629 , -0.42117172,  0.91816795],
         [ 0.50909865, -0.4464873 , -0.3630113 ]], dtype=float32)>])

Example 1.12 : create a `layer` o = w * x with ReLu activation.

In [35]:
class MyDense(layers.Layer):
    def __init__(self, input_dim, output_dim):
        super(MyDense, self).__init__()
        self.kernel = self.add_weight('w', [input_dim, output_dim], trainable=True)
    
    def call(self, inputs, training=None):
        out = inputs @ self.kernel
        out = tf.nn.relu(out)

        return out




## Customize a Net

In [36]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = MyDense(28*28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)
    
    def call(self, inputs, training=None):
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x

        

# Use Model Repositories
Example: create a model based on ResNet50

In [9]:
resnet = keras.applications.ResNet50(weights='imagenet', include_top=False)
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [13]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [11]:
x = tf.random.normal([4, 12, 12, 3])

out = resnet(x)

In [ ]:
out

# Test metrics

In [12]:
# create a new metrics
loss_meter = keras.metrics.Mean()

In [2]:
# update the loss by new data
tf.test.is_gpu_available()


True

In [3]:
tf.

SyntaxError: invalid syntax (<ipython-input-3-8683e4ff1c6e>, line 1)